In [125]:
options(digits = 17)

In [126]:
library(perm)
library(dplyr)
library(lmtest)
library(car)
library(PMCMRplus)
library(MASS)
library(rstatix)
library(psych)
library(survival)
library(survminer)
library(summarytools)
library(BSDA)
library(infer)
library(jmv)
library(DescTools)
library(sandwich)
library(afex)
library(sjstats)
library(wrassp)
library(ggpubr)
library(readxl)

In [127]:
if (!require(rstatix)) install.packages("rstatix")
if (!require(tidyr)) install.packages("tidyr")
library(rstatix)
library(tidyr)


In [128]:
data <- read_excel("C:/Users/SADEGHI/Desktop/Corr_output.xlsx", sheet="morph_av-ngl_glnu")
data1 <- data$x
data2 <- data$y
head(data)

x,y
<dbl>,<dbl>
0.120175494608009997,7775.5752257446202
0.136324072741388302,7574.0881797734764
0.144109012158208399,11203.2880765773207
0.184037759374836907,2242.9596346758499
0.126265805428844191,8607.1395184441244
0.096276950515374368,19260.2391181596213


# correlation

In [129]:
cor_base_r <- cor(data1, data2)
cor_base_r

[1] -0.26778037183472725

# Standard Deviation Difference

In [130]:
sd_data1 <- sd(data1)
sd_data2 <- sd(data2)
sd_diff <- sd_data1 - sd_data2
print(sd_diff)


[1] -7071.3156271073176


# Mean Difference

In [131]:
mean_data1 <- mean(data1)
mean_data2 <- mean(data2)
mean_diff <- mean_data1 - mean_data2
print(mean_diff)

[1] -10583.652836298226


# z_test

In [132]:
z_test <- z.test(data1, data2, alternative = "two.sided", mu = 0, sigma.x = sd(data1), sigma.y = sd(data2), conf.level = 0.95)
z_statistic <- z_test$statistic
z_p_value <- z_test$p.value
results_df <- data.frame(
  Z_Statistic = z_statistic,
  Z_P_Value = z_p_value,
  stringsAsFactors = FALSE
)

print(results_df)


          Z_Statistic               Z_P_Value
z -36.354367224151204 2.2416738430569128e-289


# Wilcoxon signed-rank test

In [133]:
wsrt_test <- wilcox.test(data1, data2, paired = TRUE)
wsrt_statistic <- wsrt_test$statistic
wsrt_p_value <- wsrt_test$p.value
results_df <- data.frame(
  WSRT_Statistic = wsrt_statistic,
  WSRT_P_Value = wsrt_p_value,
  stringsAsFactors = FALSE
)

print(results_df)

  WSRT_Statistic           WSRT_P_Value
V              0 2.5698914373803422e-98


# Levene's test

In [134]:
data_combined <- data.frame(
  value = c(data1, data2),
  group = rep(c("Group1", "Group2"), each = length(data1))
)
levene_test <- levene_test(value ~ group, data = data_combined)
lt_statistic <- levene_test$statistic
lt_p_value <- levene_test$p.value
results_df <- data.frame(
  LT_Statistic = lt_statistic,
  LT_P_Value = lt_p_value,
  stringsAsFactors = FALSE
)

print(results_df)
descriptive_stats <- data_combined %>%
  group_by(group) %>%
  summarise(
    Mean = mean(value),
    Median = median(value),
    SD = sd(value),
    Min = min(value),
    Max = max(value),
    IQR = IQR(value),
    .groups = 'drop'
  )

print(descriptive_stats)

Warning message in leveneTest.default(y = y, group = group, ...):
"group coerced to factor."
Warning message:
"Unknown or uninitialised column: `p.value`."


ERROR: Error in data.frame(LT_Statistic = lt_statistic, LT_P_Value = lt_p_value, : arguments imply differing number of rows: 1, 0


In [135]:
if (!is.null(levene_test)) {
  lt_statistic <- levene_test$statistic
  lt_p_value <- levene_test$p
}
print(lt_statistic)
print(lt_p_value)
if (!is.null(lt_statistic) & !is.null(lt_p_value)) {
  results_df <- data.frame(
    LT_Statistic = lt_statistic,
    LT_P_Value = lt_p_value,
    stringsAsFactors = FALSE
  )
  print(results_df)
} else {
  print("Levene's test failed or returned no results.")
}

[1] 464.82633982644057
[1] 3.6356980103285255e-87
        LT_Statistic             LT_P_Value
1 464.82633982644057 3.6356980103285255e-87


# Bartlett's test

In [136]:
data_combined <- data.frame(
  value = c(data1, data2),
  group = rep(c("Group1", "Group2"), each = length(data1))
)
bartlett_test <- bartlett.test(value ~ group, data = data_combined)
bt_statistic <- bartlett_test$statistic
bt_p_value <- bartlett_test$p.value
results_df <- data.frame(
  BT_Statistic = bt_statistic,
  BT_P_Value = bt_p_value,
  stringsAsFactors = FALSE
)

print(results_df)
descriptive_stats <- data_combined %>%
  group_by(group) %>%
  summarise(
    Mean = mean(value),
    Median = median(value),
    SD = sd(value),
    Min = min(value),
    Max = max(value),
    IQR = IQR(value),
    .groups = 'drop'
  )

print(descriptive_stats)

                           BT_Statistic BT_P_Value
Bartlett's K-squared 12622.189033211498          0
# A tibble: 2 × 7
  group       Mean   Median        SD    Min      Max       IQR
  <chr>      <dbl>    <dbl>     <dbl>  <dbl>    <dbl>     <dbl>
1 Group1     0.130    0.126    0.0778 0.0786     1.95    0.0289
2 Group2 10584.    8595.    7071.     3.62   53647.   7649.    


# Welch's t-test

In [137]:
data_combined <- data.frame(
  value = c(data1, data2),
  group = rep(c("Group1", "Group2"), each = length(data1))
)
shapiro_test_data1_base <- shapiro.test(data1)
shapiro_test_data2_base <- shapiro.test(data2)
shapiro_test_rstatix <- data_combined %>%
  group_by(group) %>%
  shapiro_test(value)
swt_statistic_rstatix <- shapiro_test_rstatix$statistic
swt_p_value_rstatix <- shapiro_test_rstatix$p
var_test <- var.test(data1, data2)
t_test <- t.test(data1, data2, var.equal = FALSE)
t_statistic <- t_test$statistic
t_p_value <- t_test$p.value
results_df <- data.frame(
  Group = c("Group1", "Group2"),
  W_Statistic_BaseR = c(shapiro_test_data1_base$statistic, shapiro_test_data2_base$statistic),
  P_Value_BaseR = c(shapiro_test_data1_base$p.value, shapiro_test_data2_base$p.value),
  W_Statistic_rstatix = c(swt_statistic_rstatix[1], swt_statistic_rstatix[2]),
  P_Value_rstatix = c(swt_p_value_rstatix[1], swt_p_value_rstatix[2]),
  F_Statistic = var_test$statistic,
  F_P_Value = var_test$p.value,
  T_Statistic = t_statistic,
  T_P_Value = t_p_value,
  stringsAsFactors = FALSE
)

print(results_df)

Warning message in data.frame(Group = c("Group1", "Group2"), W_Statistic_BaseR = c(shapiro_test_data1_base$statistic, :
"row names were found from a short variable and have been discarded"


   Group   W_Statistic_BaseR          P_Value_BaseR W_Statistic_rstatix
1 Group1 0.15662734413787349 4.5038078170791347e-45 0.15662734413787349
2 Group2 0.83827358893124349 5.1038240299305787e-24 0.83827358893124349
         P_Value_rstatix            F_Statistic F_P_Value         T_Statistic
1 4.5038078170791347e-45 1.2110676185900462e-10         0 -36.354367224151197
2 5.1038240299305787e-24 1.2110676185900462e-10         0 -36.354367224151197
                T_P_Value
1 1.2251276114129946e-152
2 1.2251276114129946e-152


# Permutation test

In [138]:
test_statistic <- function(x, y) {
  return(mean(x) - mean(y))
}
obs_stat <- test_statistic(data1, data2)
n_permutations <- 1000
perm_stats <- numeric(n_permutations)
combined <- c(data1, data2)
for (i in 1:n_permutations) {
  permuted <- sample(combined)
  perm_data1 <- permuted[1:length(data1)]
  perm_data2 <- permuted[(length(data1)+1):length(combined)]
  perm_stats[i] <- test_statistic(perm_data1, perm_data2)
}
p_value <- mean(abs(perm_stats) >= abs(obs_stat))
print(paste("pt_statistic:", obs_stat))
print(paste("pt_p_value:", p_value))


[1] "pt_statistic: -10583.6528362982"
[1] "pt_p_value: 0"


In [139]:
if (!require(infer)) install.packages("infer")
library(infer)
data <- data.frame(
  value = c(data1, data2),
  group = factor(rep(c("data1", "data2"), each = length(data1)))
)
perm_test <- data %>%
  specify(response = value, explanatory = group) %>%
  hypothesize(null = "independence") %>%
  generate(reps = 1000, type = "permute") %>%
  calculate(stat = "diff in means")
p_value <- perm_test %>%
  get_p_value(obs_stat = obs_stat, direction = "two.sided")
print(paste("pt_statistic:", obs_stat))
print(paste("pt_p_value:", p_value))


Warning message:
"The statistic is based on a difference or ratio; by default, for difference-based statistics, the explanatory variable
is subtracted in the order "data1" - "data2", or divided in the order "data1" / "data2" for ratio-based statistics. To
specify this order yourself, supply `order = c("data1", "data2")` to the calculate() function."
Warning message:
"Please be cautious in reporting a p-value of 0. This result is an approximation based on the number of `reps` chosen in
the `generate()` step.
ℹ See `get_p_value()` (`?infer::get_p_value()`) for more information."


[1] "pt_statistic: -10583.6528362982"
[1] "pt_p_value: 0"


# F-test

In [140]:
data <- data.frame(
  value = c(data1, data2),
  group = factor(rep(c("data1", "data2"), each = length(data1)))
)
anova_result <- aov(value ~ group, data = data)
summary_result <- summary(anova_result)
F_statistic <- summary_result[[1]]$`F value`[1]
F_p_value <- summary_result[[1]]$`Pr(>F)`[1]

print(paste("F_statistic:", F_statistic))
print(paste("F_p_value:", F_p_value))


[1] "F_statistic: 1321.64001626847"
[1] "F_p_value: 1.14571616167022e-194"


# Shapiro-Wilk Test

In [141]:
shapiro_test_data1_base <- shapiro.test(data1)
shapiro_test_data2_base <- shapiro.test(data2)
results_df <- data.frame(
  Group = c("Group1", "Group2"),
  W_Statistic_BaseR = c(shapiro_test_data1_base$statistic, shapiro_test_data2_base$statistic),
  P_Value_BaseR = c(shapiro_test_data1_base$p.value, shapiro_test_data2_base$p.value),
  stringsAsFactors = FALSE
)
print(results_df)

   Group   W_Statistic_BaseR          P_Value_BaseR
1 Group1 0.15662734413787349 4.5038078170791347e-45
2 Group2 0.83827358893124349 5.1038240299305787e-24


# Mann-Whitney U Test

In [142]:
data_combined <- data.frame(
  value = c(data1, data2),
  group = rep(c("Group1", "Group2"), each = length(data1))
)

# Base R (stats)
mann_whitney_base_r <- wilcox.test(value ~ group, data = data_combined)
mwut_statistic_base_r <- mann_whitney_base_r$statistic
mwut_p_value_base_r <- mann_whitney_base_r$p.value
mwut_statistic_base_r
mwut_p_value_base_r

W 
0

[1] 2.6102454234731252e-194

# Kruskal-Wallis Test

In [143]:
# Base R (stats)
kruskal_wallis_base_r <- kruskal.test(value ~ group, data = data_combined)
kwt_statistic_base_r <- kruskal_wallis_base_r$statistic
kwt_p_value_base_r <- kruskal_wallis_base_r$p.value
kwt_statistic_base_r
kwt_p_value_base_r

Kruskal-Wallis chi-squared 
         884.2506350550384

[1] 2.6036157867038673e-194

In [144]:
# rstatix package
kwt_rstatix <- data_combined %>%
  kruskal_test(value ~ group)
kwt_statistic_rstatix <- kwt_rstatix$statistic
kwt_p_value_rstatix <- kwt_rstatix$p
kwt_statistic_rstatix
kwt_p_value_rstatix

Kruskal-Wallis chi-squared 
         884.2506350550384

[1] 2.5999999999999991e-194

# ANOVA

In [145]:
library(dplyr)
library(car)
library(lmtest)
library(sandwich)
library(BSDA)
data_combined <- data.frame(
  group = factor(rep(c("Group1", "Group2"), each = length(data1))),
  value = c(data1, data2)
)

# Base R (stats) - ANOVA
anova_base_r <- aov(value ~ group, data = data_combined)
ANOVA_F_statistic_base_r <- summary(anova_base_r)[[1]]["group", "F value"]
ANOVA_p_value_base_r <- summary(anova_base_r)[[1]]["group", "Pr(>F)"]

# car package - ANOVA
anova_car <- Anova(anova_base_r, type = "II")
ANOVA_F_statistic_car <- anova_car["group", "F value"]
ANOVA_p_value_car <- anova_car["group", "Pr(>F)"]

# lmtest package - ANOVA 
anova_lmtest <- waldtest(anova_base_r)
ANOVA_F_statistic_lmtest <- anova_lmtest[2, "F"]
ANOVA_p_value_lmtest <- anova_lmtest[2, "Pr(>F)"]

# BSDA package - ANOVA 
ANOVA_F_statistic_bsda <- NA
ANOVA_p_value_bsda <- NA

results_list_ANOVA_F_statistic <- list(
  Base_R_stats = ANOVA_F_statistic_base_r,
  car = ANOVA_F_statistic_car,
  lmtest = ANOVA_F_statistic_lmtest,
  BSDA = ANOVA_F_statistic_bsda
)

results_list_ANOVA_p_value <- list(
  Base_R_stats = ANOVA_p_value_base_r,
  car = ANOVA_p_value_car,
  lmtest = ANOVA_p_value_lmtest,
  BSDA = ANOVA_p_value_bsda
)
length_F_statistic_ANOVA <- length(results_list_ANOVA_F_statistic)
length_p_value_ANOVA <- length(results_list_ANOVA_p_value)

if (length_F_statistic_ANOVA != length_p_value_ANOVA) {
  stop("Mismatch in lengths of results lists: F-statistic list has ", 
       length_F_statistic_ANOVA, " elements, but P-value list has ", length_p_value_ANOVA, " elements.")
}

results_df_ANOVA <- data.frame(
  package = names(results_list_ANOVA_F_statistic),
  F_Statistic = unlist(results_list_ANOVA_F_statistic),
  P_Value = unlist(results_list_ANOVA_p_value),
  stringsAsFactors = FALSE
)
print("ANOVA F-Statistics Results:")
print(results_list_ANOVA_F_statistic)
print("ANOVA P-Values Results:")
print(results_list_ANOVA_p_value)
print("ANOVA Summary Data Frame:")
print(results_df_ANOVA)


[1] "ANOVA F-Statistics Results:"
$Base_R_stats
[1] 1321.6400162684724

$car
[1] 1321.6400162684727

$lmtest
[1] 1321.6400162684724

$BSDA
[1] NA

[1] "ANOVA P-Values Results:"
$Base_R_stats
[1] 1.1457161616702174e-194

$car
[1] 1.1457161616700872e-194

$lmtest
[1] 1.1457161616702174e-194

$BSDA
[1] NA

[1] "ANOVA Summary Data Frame:"
                  package        F_Statistic                 P_Value
Base_R_stats Base_R_stats 1321.6400162684724 1.1457161616702174e-194
car                   car 1321.6400162684727 1.1457161616700872e-194
lmtest             lmtest 1321.6400162684724 1.1457161616702174e-194
BSDA                 BSDA                 NA                      NA


# Chi-Square Test

In [146]:
data_combined <- data.frame(
  value = c(data1, data2),
  group = rep(c("Group1", "Group2"), each = length(data1))
)
data_combined$category <- cut(data_combined$value, breaks = 5)

contingency_table <- table(data_combined$group, data_combined$category)

chi2_test <- chisq.test(contingency_table)
chi2_statistic <- chi2_test$statistic
chi2_p_value <- chi2_test$p.value
results_df <- data.frame(
  Chi2_Statistic = chi2_statistic,
  Chi2_P_Value = chi2_p_value,
  stringsAsFactors = FALSE
)
print(results_df)


Warning message in chisq.test(contingency_table):
"Chi-squared approximation may be incorrect"


              Chi2_Statistic           Chi2_P_Value
X-squared 271.92909280500521 1.2244919277578654e-57


# Kolmogorov-Smirnov Test

In [147]:
data_combined <- data.frame(
  value1 = data1,
  value2 = data2
)
kst_statistic_base_r <- ks.test(data1, data2)$statistic
kst_p_value_base_r <- ks.test(data1, data2)$p.value
kst_statistic_base_r
kst_p_value_base_r 

D 
1

[1] 0

# Paired t-test

In [148]:
differences <- data1 - data2
mean_diff <- mean(differences)
sd_diff <- sd(differences)
n <- length(differences)
t_statistic <- mean_diff / (sd_diff / sqrt(n))

p_value <- 2 * pt(-abs(t_statistic), df = n - 1)

t_statistic
p_value


[1] -36.354260092547435

[1] 1.2266005768430398e-152

In [149]:
data_combined <- data.frame(
  value = c(data1, data2),
  group = rep(c("Group1", "Group2"), each = length(data1))
)

# Base R (stats)
itt_stats <- t.test(data1, data2, var.equal = TRUE)
itt_statistic_base_r <- itt_stats$statistic
itt_p_value_base_r <- itt_stats$p.value

# lmtest package
model_lm <- lm(value ~ group, data = data_combined)
itt_lmtest <- coeftest(model_lm, vcov = vcovHC(model_lm, type = "HC1"))[2,]
itt_statistic_lmtest <- itt_lmtest["t value"]
itt_p_value_lmtest <- 2 * (1 - pnorm(abs(itt_statistic_lmtest))) # p-value calculation should use absolute value


itt_statistic_base_r 
itt_p_value_base_r
itt_statistic_lmtest
itt_p_value_lmtest

t 
-36.354367224151204

[1] 1.1457161616790812e-194

t value 
36.354367224151304

t value 
      0